# Логистическая регрессия. Практика

В этом задании вам предлагается спрогнозировать, купит клиент велосипед или нет, обучив логистическую регрессию.

In [69]:
# подключим библиотеки
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

In [70]:
# считаем данные
data = pd.read_csv('https://raw.githubusercontent.com/evgpat/edu_stepik_practical_ml/main/datasets/bike_buyers_clean.csv')

In [71]:
# выводим первые 5 строк датафрейма
data.head(5)

,ID,Marital Status,Gender,Income,Children,Education,Occupation,Home Owner,Cars,Commute Distance,Region,Age,Purchased Bike
0,12496,Married,Female,40000,1,Bachelors,Skilled Manual,Yes,0,0-1 Miles,Europe,42,No
1,24107,Married,Male,30000,3,Partial College,Clerical,Yes,1,0-1 Miles,Europe,43,No
2,14177,Married,Male,80000,5,Partial College,Professional,No,2,2-5 Miles,Europe,60,No
3,24381,Single,Male,70000,0,Bachelors,Professional,Yes,1,5-10 Miles,Pacific,41,Yes
4,25597,Single,Male,30000,0,Bachelors,Clerical,No,0,0-1 Miles,Europe,36,Yes


In [72]:
# смотрим размер датафрейма
data.shape

(1000, 13)

Выведите статистики по категориальным признакам, чтобы посмотреть, сколько категорий в каждом категориальном (нечисловом) признаке.

Для этого можно воспользоваться методом `describe` из библиотеки pandas со значением параметра `include = 'object'`.

**Вопрос:** в каком категориальном признаке встречаются три различных значения?

In [73]:
data.describe(include='object')

,Marital Status,Gender,Education,Occupation,Home Owner,Commute Distance,Region,Purchased Bike
count,1000,1000,1000,1000,1000,1000,1000,1000
unique,2,2,5,5,2,5,3,2
top,Married,Male,Bachelors,Professional,Yes,0-1 Miles,North America,No
freq,539,509,306,276,685,366,508,519


Закодируйте все категориальные столбцы с двумя категориями следующим образом:  
самая часто встречающаяся категория превращается в 1, другая в 0.

In [74]:
data['Marital Status'] = data['Marital Status'].map({'Married' : 1, 'Single' : 0})
data['Gender'] = data['Gender'].map({'Male' : 1, 'Female' : 0})
data['Home Owner'] = data['Home Owner'].map({'Yes' : 1, 'No' : 0})
data['Purchased Bike'] = data['Purchased Bike'].map({'No': 1,'Yes': 0})

Удалите остальные категориальные столбцы.

**Вопрос:** сколько категориальных столбцов вы удалили?

In [75]:
data = data.drop(['Education', 'Occupation', 'Commute Distance', 'Region'], axis = 1)

Удалите столбец `ID`, так как он по сути нечисловой.

In [76]:
data = data.drop(['ID'], axis = 1)

Сформируйте матрицу объект-признак `X` и вектор `y` с целевой переменной.  
Целевая переменная - это последний столбец, `Purchased Bike`.

In [77]:
X, y = data.drop('Purchased Bike', axis=1), data['Purchased Bike']

Разбейте данные на тренировочную и тестовую часть (`Xtrain`, `Xtest`, `ytrain`, `ytest`), в тест отправьте 30% данных.  
Зафиксируйте `random_state = 42`.

In [78]:
from sklearn.model_selection import train_test_split

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.3, random_state=42)

**Вопрос:** сколько объектов в матрице `Xtrain`?

In [79]:
print(f'{Xtrain.shape[0]} объектов')

700 объектов


Почти всё готово для обучения модели!

Осталось отмасштабировать матрицу `X`, так как линейные модели чувствительны к масштабу данных.

*  Обучите на тренировочной матрице (`Xtrain`) `MinMaxScaler` из библиотеки `sklearn.preprocessing`
*  Примените масштабирование и к `Xtrain`, и к `Xtest`
*  Переведите полученные после масштабирования `np.array` обратно в pandas `dataframe`.

Полученные масштабированные матрицы назовите, как и раньше, `Xtrain` и `Xtest`.

In [80]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
Xtrain = pd.DataFrame(scaler.fit_transform(Xtrain), columns=X.columns)
Xtest = pd.DataFrame(scaler.transform(Xtest), columns=X.columns)


Теперь обучите логистическую регрессию на тренировочных данных

Сделайте предсказания на тренировочных и на тестовых данных.

Оцените значение accuracy на трейне и на тесте.

In [81]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

model = LogisticRegression()
model.fit(Xtrain, ytrain)
ypred = model.predict(Xtest)

print(f'Accurasy: {accuracy_score(ytest, ypred).round(2)}')

Accurasy: 0.58


In [82]:
print(Xtest.shape, Xtrain.shape)

(300, 7) (700, 7)


Качество модели получилось невысоким, зато модель не переобучена.

Попробуем добавить новых признаков в модель, используя [PolynomialFeatures](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html).

Создайте полиномиальные признаки degree = 2.

Как обычно:
*  `fit` делайте на тренировочных данных
*  `transform` и на тренировочных, и на тестовых данных. Затем верните результат к формату pandas `dataframe`.

Полученные матрицы назовите, как и раньше, `Xtrain` и `Xtest`.

In [83]:
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(degree=2)
Xtrain_poly = pd.DataFrame(poly.fit_transform(Xtrain))
Xtest_poly = pd.DataFrame(poly.transform(Xtest))


**Вопрос:** на сколько признаков стало больше при добавлении полиномиальных признаков второй степени?

In [87]:
print(f'Добавилось {Xtrain_poly.shape[1] - Xtrain.shape[1]} признаков')

Добавилось 29 признаков


Заново обучите логистическую регрессию, уже на расширенной матрице признаков, и сделайте предсказания на трейне и тесте, а затем оцените качество (*accuracy*).

In [88]:
model.fit(Xtrain_poly, ytrain)
ypred = model.predict(Xtest_poly)
print(f'Accurasy: {accuracy_score(ytest, ypred).round(2)}')

Accurasy: 0.62


**Вопрос:** на сколько повысилось качество модели на тестовых данных?  
Ответ округлите до сотых.

**Ответ:** на 0.04

Появились новые требования от заказчика!

Заказчик просит, чтобы:
*  доля найденных моделью потенциальных покупателей была максимальной
*  accuracy при этом была не ниже, чем 0.6 (отклонения *accuracy* на тестовых данных на $\pm 0.05$ допустимы).

Сначала посмотрите, какие значения *recall* и *accuracy* имеют предсказния модели на тесте с классами, предсказанными по умолчанию (методом `predict`).

In [93]:
from sklearn.metrics import recall_score

print(f'Recall: {recall_score(ytest, ypred).round(3)}')
print(f'Accurasy: {accuracy_score(ytest, ypred).round(2)}')

Recall: 0.696
Accurasy: 0.62


Подберём порог для перевода вероятностей в классы, чтобы оптимизировать требуемые метрики!

Разобъем тренировочные данные на трейн и валидацию, чтобы по валидационной части подбирать порог.

In [94]:
XtrainS, Xval, ytrainS, yval = train_test_split(Xtrain, ytrain, test_size=0.3, random_state=42)

XtrainS.shape, Xval.shape

((490, 7), (210, 7))

* Обучите модель на тренировочных данных.
* Предскажите вероятности положительного класса на валидационных данных

В цикле для каждого значения порога:
*  переведите вероятности в классы
*  вычислите полноту (на валидационных данных)

Выведите на экран:

1) значение порога, дающее максимальный *recall*, при условии, что *accuracy* $\geq$ 0.6.

2) значение *recall* при этом пороге

3) значение *accuracy* для этого порога


Ищите порог на отрезке от 0 до 1 с шагом 0.01.

In [108]:
from sklearn.metrics import recall_score

model = LogisticRegression()
model.fit(XtrainS, ytrainS)

RecMax = -1
BestThr = -1
BestAcc = -1

for thr in np.arange(0, 1, 0.01):
    ypred = (model.predict_proba(Xval)[:, 1] > thr).astype(int)
    rec = recall_score(yval, ypred)
    acc = accuracy_score(yval, ypred)
    if rec > RecMax and acc >= 0.6:
        RecMax = rec
        BestThr = thr
        BestAcc = acc
    # ваш код здесь

print(f'Лучший порог для перевода вероятностей в классы {BestThr}')
print(f'Наилучший recall {RecMax.round(3)} при условии accuracy >= 0.6')
print(f'Оптимальный accuracy {BestAcc}')

Лучший порог для перевода вероятностей в классы 0.43
Наилучший recall 0.824 при условии accuracy >= 0.6
Оптимальный accuracy 0.6


Теперь заново обучите модель на исходных тренировочных данных (`Xtrain`, `ytrain`), предскажите вероятности на тесте и переведите их в классы по найденному порогу.

**Вопрос:** какое значение *recall* получилось на тестовых данных после подбора порога?  

In [119]:
model.fit(Xtrain, ytrain)
ypred = (model.predict_proba(Xtest)[ : , 1] > BestThr).astype(int)

print(' Итого:')
print(f' Recall = {recall_score(ytest, ypred).round(3)}')
print(f' Accuracy = {accuracy_score(ytest, ypred).round(3)}')

 Итого:
 Recall = 0.811
 Accuracy = 0.573


При помощи подбора порога удалось сильно увеличить значение *recall*!  
Однако, как видно, на тесте не удалось сохранить условие $accuracy \geq 0.6$ (но в допустимые рамки уложились!)

Это свидетельство небольшого переобучения модели. Однако в сухом остатке имеет значительное увеличение полноты, что является приоритетом для заказчика.